<h3>Compute RSR values</h3>
<p>
    This notebook contains the function "getRSR" which is the function used to obtain RSR values for analysis. The notebook also contains the function "getParameters", which returns the full model and was what we used to compute the model convergence figures in the supplement.
</p>

In [1]:
import numpy as np

In [2]:
# Helpers for optimization

def raiseP(ps, n, problems):
    ps_spread = np.matmul( ps.reshape(-1,1), np.ones((1,n)), dtype = np.float)
    ps_pow = ps_spread ** np.array([i for i in range(n)], dtype = np.float)
    for i in problems:
        ps_pow[i][0] = 1
    return ps_pow

def evaluate(counts, multiplicity, cs, ps, alphas, problems):
    lambdas = ( (raiseP(ps, len(alphas), problems) * alphas).transpose() * np.exp(cs) ).transpose()
    for i in problems:
        for j in range(len(alphas)):
            lambdas[i][j] = 1
            
    lgl = np.log(lambdas) * counts
    for i in problems:
        for j in range(1, len(alphas)):
            lambdas[i][j] = 0
            
    p_loss = np.sum(np.sum(lgl - lambdas, axis = 1) * multiplicity)
    c_loss = np.sum(multiplicity * (cs ** 2)/2)
    return p_loss - c_loss

epsilon = 10 ** (-8)

def optimizeP(s_xcount, cs, ps, alphas):
    scale = np.array([i for i in range(len(alphas))], dtype = np.float)
    leader = np.array([1] + ([0]*(len(alphas)-1)), dtype = np.float).reshape(1,-1)
    polys = np.flip(np.matmul(s_xcount.reshape(-1,1), leader)
                    - np.matmul(np.exp(cs).reshape(-1,1), (alphas*scale).reshape(1,-1)), axis = 1)
    
    ps2 = []
    for i,poly in enumerate(polys):
        roots = np.roots(poly)
        newval = max( [r.real for r in roots if abs(r.imag) < epsilon and r.real > 0.0] + [0.0] )
        ps2.append(newval)
    return np.array(ps2)

def optimizeAlphas(counts, multiplicity, cs, ps, alphas, problems):
    ps_pow = raiseP(ps, len(alphas), problems)
    denoms = np.sum( ps_pow.transpose() * (np.exp(cs) * multiplicity), axis = 1)
    nums = np.sum(counts.transpose() * multiplicity, axis = 1)
    return nums/denoms

def newton(ci, term, target, iterations):
    c = ci
    for i in range(iterations):
        cterm = np.exp(c) * term
        current = cterm + c
        deriv = cterm + 1
        rise = target-current
        run = rise/deriv
        c = np.clip( c + run, -15, 15 )
    return c
        
def optimizeC(s_count, cs, ps, alphas, problems):
    ps_pow = raiseP(ps, len(alphas), problems)
    expterm = np.sum(ps_pow * alphas, axis = 1)
    newc = newton(cs, expterm, s_count, 25)
    return newc

def precomputeStats(read_counts):
    s_xcount = np.sum(read_counts * np.array([i for i in range(read_counts.shape[1])]), axis = 1)
    s_count = np.sum(read_counts, axis = 1)
    return s_xcount, s_count

In [3]:
# Function for obtaining the parameters that fit the RSR model
# Use this if you want to compute all the parameters

# data: A list of tuples (multiplicity, read vector)
# iterations: Number of iterations to run for

def getParameters(data, iterations):
    counts = []
    multiplicities = []
    problems = []
    for i, (multiplicity, count) in enumerate(data):
        multiplicities.append(multiplicity)
        counts.append(count)
        if sum(count[1:]) == 0:
            problems.append(i)
    counts = np.array(counts, dtype = np.float)
    multiplicities = np.array(multiplicities, dtype = np.float)
    
    s_xcount, s_count = precomputeStats(counts)
    alphas = np.array([1.0] * len(counts[0]), dtype = np.float)
    cs = np.random.normal(0,1,len(multiplicities))
    ps = np.random.uniform(0.1,1,len(multiplicities))
    for i in problems: ps[i] = 0
    
    deltas = []
    valueTrace = []
    valueTrace.append(evaluate(counts, multiplicities, cs, ps, alphas, problems))
    for iteration in range(iterations):
        alphas = optimizeAlphas(counts, multiplicities, cs, ps, alphas, problems)
        cs = optimizeC(s_count, cs, ps, alphas, problems)
        ps = optimizeP(s_xcount, cs, ps, alphas)
        for i in problems: ps[i] = 0
        valueTrace.append(evaluate(counts, multiplicities, cs, ps, alphas, problems))
        print ("Iteration {}: score = {}".format(iteration+1, valueTrace[-1]))
        
    rekey = [x for x in zip([tuple([int(z) for z in row]) for row in counts], ps, cs)]
    return rekey, alphas, valueTrace

In [4]:
# Wrapper for just getting the RSR values

# data: A dictionary that maps sequences to read vectors
# iterations: Number of iterations to run for
# returns: A dictionary that maps sequences to RSR values

def getRSR(data, iterations = 500):
    counter = {}
    for k in data:
        v = tuple(data[k])
        if v not in counter:
            counter[v] = 1
        else:
            counter[v] += 1
    pass_data = []
    for k,v in counter.items():
        pass_data.append((v,k))
        
    rekey,_,_ = getParameters(pass_data, iterations)
    rvtoRSR = {}
    for rv, RSR, _ in rekey:
        rvtoRSR[rv] = RSR
    
    output = {}
    for k in data:
        output[k] = rvtoRSR[tuple(data[k])]
        
    return output